In [1]:
import imp
import gym
from gym import spaces
import numpy as np
import os
import random
import uuid

import ray
from ray import air, tune
from ray.rllib.algorithms import ppo
from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork
from ray.rllib.utils.framework import try_import_tf, try_import_torch

from environment.multi_agent_grid import MultiAgentGrid
from model.custom_model import CustomModel

tf1, tf, tfv = try_import_tf()

/tmp/ipykernel_2727/932753981.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/paul/ray/venv/lib/python3.8/site-packages/requests/__init__.py:43: DeprecationWarning: 'urllib3[secure]' extra is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  import urllib3
/home/paul/ray/venv/lib/python3.8/site-packages/botocore/httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext


In [2]:

if ray.is_initialized():
    ray.shutdown()

ModelCatalog.register_custom_model("my_model", CustomModel)
ENV_SIZE = 8

config = {
    "env": MultiAgentGrid,  # or "corridor" if registered above
    "env_config": {
        "render_mode": 'none',  # 'all', 'first', or 'none'
        "size": ENV_SIZE,
        "n_robots": 2
    },
    "render_env": False,
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "lr": 0.0001,
    #"lr": 0.001,
    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
    "model": {
        "custom_model": "my_model",
        "vf_share_layers": True,
        "fcnet_hiddens": [128],
        #"fcnet_hiddens": [64]
    },
    "evaluation_config": {
        # Render the env while evaluating.
        # Note that this will always only render the 1st RolloutWorker's
        # env and only the 1st sub-env in a vectorized env.
    },
    "num_workers": 8,  # parallelism
    "framework": "tf2",
    "disable_env_checking": True
}

stop = {
    "training_iteration": 10,
    "timesteps_total": 10000
}

print("Training automatically with Ray Tune")

ray.init(include_dashboard=True, num_gpus=8)
tuner = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=air.RunConfig(stop=stop),
    tune_config=tune.TuneConfig(num_samples=1)
)
results = tuner.fit()

Training automatically with Ray Tune
Usage stats collection is enabled. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


KeyboardInterrupt: 

In [ ]:
from copy import deepcopy
from ray.rllib.agents.ppo import PPOTrainer
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")
checkpoint = best_result.checkpoint

trainer_cfg = {
    'env': best_result.config['env'],
    'env_config': best_result.config['env_config'],
    'lr': best_result.config['lr'],
    'model': deepcopy(best_result.config['model']),
    'framework': best_result.config['framework'],
}

trainer = PPOTrainer(config=trainer_cfg)
trainer.restore(checkpoint)

/home/paul/ray/venv/lib/python3.8/site-packages/ray/_private/ray_option_utils.py:290: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/master/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
(pid=10244) /home/paul/ray/venv/lib/python3.8/site-packages/botocore/httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
(pid=10244)   from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext
(pid=10245) /home/paul/ray/venv/lib/python3.8/site-packages/botocore/httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3

(RolloutWorker pid=10244) ENV CONFIG----  {'render_mode': 'none', 'size': 8, 'n_robots': 2, worker=1/2, vector_idx=0, remote=False}
(RolloutWorker pid=10244) Model initialization: obs_space=Box(-1.0, 1.0, (6,), float32), action_space=MultiDiscrete([5 5])
(RolloutWorker pid=10244) Initializing FC Network: obs_space=Box(-1.0, 1.0, (6,), float32),  action_space=MultiDiscrete([5 5]), num_outputs=10, model_config={'_use_default_native_models': False, '_disable_preprocessor_api': False, '_disable_action_flattening': False, 'fcnet_hiddens': [128], 'fcnet_activation': 'tanh', 'conv_filters': None, 'conv_activation': 'relu', 'post_fcnet_hiddens': [], 'post_fcnet_activation': 'relu', 'free_log_std': False, 'no_final_linear': False, 'vf_share_layers': True, 'use_lstm': False, 'max_seq_len': 20, 'lstm_cell_size': 256, 'lstm_use_prev_action': False, 'lstm_use_prev_reward': False, '_time_major': False, 'use_attention': False, 'attention_num_transformer_units': 1, 'attention_dim': 64, 'attention_num_

In [ ]:
env_cfg = EnvContext(env_config=best_result.config['env_config'], worker_index=0, num_workers=1)
env_cfg['render_mode'] = 'all'

env = MultiAgentGrid(config=env_cfg)

ENV CONFIG----  {'render_mode': 'all', 'size': 8, 'n_robots': 2, worker=0/1, vector_idx=0, remote=False}


In [ ]:
n_record = 50

observation = env.reset()
done = False
n_recorded = 0
while n_recorded < n_record:
  env.render()
  action = trainer.compute_single_action(observation)
  observation, reward, done, info = env.step(action)
  if done:
    print(f"Completed run {n_recorded}")
    env.reset()
    n_recorded += 1
env.close()

error: font not initialized